In [2]:
import nltk

nltk.download([
    "names",
    "stopwords",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
    ])

     ---------------------------------------- 1.5/1.5 MB 7.9 MB/s eta 0:00:00
     -------------------------------------- 267.7/267.7 KB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 KB 5.8 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\65831\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\names.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [138]:
import pandas as pd
import numpy as np
from datetime import datetime
import json

import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

import string
import re

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:
df_world= pd.read_csv('../Data/News/Global/world_news.csv')
df_politics= pd.read_csv('../Data/News/Global/politics_news.csv')
df_coronavirus= pd.read_csv('../Data/News/Global/coronavirus_news.csv')
df_aapl= pd.read_csv('../Data/News/Stock/aapl_news.csv')
df_meta= pd.read_csv('../Data/News/Stock/meta_news.csv')
df_tsla= pd.read_csv('../Data/News/Stock/tsla_news.csv')
print('world shape ', df_world.shape)
print('politics shape ',df_politics.shape)
print('coronavirus shape ',df_coronavirus.shape)
print('aapl shape ',df_aapl.shape)
print('meta shape ', df_meta.shape)
print('tsla shape ', df_tsla.shape)
print(df_aapl.head())

world shape  (2239, 3)
politics shape  (887, 3)
coronavirus shape  (105, 3)
aapl shape  (2289, 3)
meta shape  (2465, 3)
tsla shape  (1776, 3)
           Date                                              Title  \
0  Aug 31, 2022  Top 5 Things to Watch in Markets in the Week A...   
1  Aug 31, 2022  Dow Futures Move Higher As Investors Brace for...   
2  Aug 30, 2022  China's 20th Communist Party Congress: What yo...   
3  Aug 30, 2022  Top 5 Things to Watch in Markets in the Week A...   
4  Aug 30, 2022  Dow Futures Move Higher As Investors Brace for...   

                                                Text  
0  By Yasin Ebrahim\r\nInvesting.com -- The Dow s...  
1  By Yasin Ebrahim\r\nInvesting.com -- The S&P 5...  
2  By Sam Boughedda\r\nAhead of Broadcom's (NASDA...  
3  By Yasin Ebrahim\r\nInvesting.com -- The Dow c...  
4  By Liz Moyer\r\nInvesting.com -- Stocks fell f...  


In [6]:
def remove_irrelevant_content(text):
    headline_only_string = "This headline-only article is meant to show you why a stock is moving, the most difficult aspect of stock trading"

    if headline_only_string in text:
        return ""
    else:
        return text

def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

def lowercase(text):
    return text.lower()

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

def remove_special_character(text):
    text = text.replace('\n', ' ') 
    return text

def lemmatize(text):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in text]
    return corpus

def preprocess_text(text):
    text = remove_irrelevant_content(text)
    text = remove_punctuation(text)
    text = lowercase(text)
    # text = remove_stopwords(text)
    text = remove_special_character(text)
    # text = lemmatize(text)
    return text

In [62]:
sia = SentimentIntensityAnalyzer()

def vader_process(df) : 
    df['Date'] = pd.to_datetime(df['Date'])
    df['Processed Title'] = df['Title'].apply(lambda x: preprocess_text(x))
    df['Processed Text'] = df['Text'].apply(lambda x: preprocess_text(x))
    df['Sentiment Title'] = df['Processed Title'].apply(lambda x: sia.polarity_scores(x))
    df['Sentiment Text'] = df['Processed Text'].apply(lambda x: sia.polarity_scores(x))
    # df['Positive Title'] = df['Sentiment Title'].apply(lambda x: x["compound"] > 0)
    # df['Positive Text'] = df['Sentiment Text'].apply(lambda x: x["compound"] > 0)
    return df

In [148]:
def sentiment_aggregator(df, title = True, type="mean"):
    """
    Aggregates sentiments on a per day basis.

    Parameters
    ----------
    df: DataFrame
        Dataset generated after sentiment analysis.
    title: boolean
        To indicate if the news title or news body text is used to generate the aggregated sentiment. 
        Default is True (ie. News title is used for aggregated sentiment)
    type: Str {"mean", "abs_max"}
        To indicate method of calculation.
        "mean": Group by Date and takes mean of "Compound"
        "abs_max": Calculates the absolute max of "Positive" and "Negative" column. Then group by Date and takes mean of this new column

    Returns
    -------
    Output : Series
        Contains aggregated sentiment for each day
    """
   
    target = "Sentiment Title" if title else "Sentiment Text"
    
    df[target] = df[target].str.replace('\'','\"')
    df[target] = df[target].apply(lambda x: json.loads(x))

    df['Negative'] = df[target].apply(lambda x: x.get('neg'))
    df['Neutral'] = df[target].apply(lambda x: x.get('neu'))
    df['Positive'] = df[target].apply(lambda x: x.get('pos'))
    df['Compound'] = df[target].apply(lambda x: x.get('compound'))

    if type == "mean":
        return df.groupby('Date')['Compound'].aggregate('mean')

    elif type == "abs_max":
        df['Negative'] = df[target].apply(lambda x: -x.get('neg'))
        df['Sentiment'] = df.apply(lambda x: max(x['Negative'], x['Positive'], key=abs), axis=1)

        return df.groupby('Date')['Sentiment'].aggregate('mean')


In [149]:
def vader_all():
    df_world= pd.read_csv('../Data/News/Global/world_news.csv')
    df_politics= pd.read_csv('../Data/News/Global/politics_news.csv')
    df_coronavirus= pd.read_csv('../Data/News/Global/coronavirus_news.csv')
    df_aapl= pd.read_csv('../Data/News/Stock/aapl_news.csv')
    df_meta= pd.read_csv('../Data/News/Stock/meta_news.csv')
    df_tsla= pd.read_csv('../Data/News/Stock/tsla_news.csv')
    
    df_aapl = vader_process(df_aapl)
    df_meta = vader_process(df_meta)
    df_tsla = vader_process(df_tsla)
    df_world = vader_process(df_world)
    df_politics = vader_process(df_politics)
    df_coronavirus = vader_process(df_coronavirus)

    df_aapl.to_csv('../Data-Processed/News/Stock/aapl_vader.csv',index=False)
    df_meta.to_csv('../Data-Processed/News/Stock/meta_vader.csv',index=False)
    df_tsla.to_csv('../Data-Processed/News/Stock/tsla_vader.csv',index=False)
    df_world.to_csv('../Data-Processed/News/Global/world_vader.csv',index=False)
    df_politics.to_csv('../Data-Processed/News/Global/politics_vader.csv',index=False)
    df_coronavirus.to_csv('../Data-Processed/News/Global/coronavirus_vader.csv',index=False)
    

def aggregate_sentiment_all(title, type):
    df_world_vader= pd.read_csv('../Data-Processed/News/Global/world_vader.csv')
    df_politics_vader= pd.read_csv('../Data-Processed/News/Global/politics_vader.csv')
    df_coronavirus_vader= pd.read_csv('../Data-Processed/News/Global/coronavirus_vader.csv')
    df_aapl_vader= pd.read_csv('../Data-Processed/News/Stock/aapl_vader.csv')
    df_meta_vader= pd.read_csv('../Data-Processed/News/Stock/meta_vader.csv')
    df_tsla_vader= pd.read_csv('../Data-Processed/News/Stock/tsla_vader.csv')

    aggregated_sentiment_aapl = sentiment_aggregator(df_aapl_vader, title=title, type=type)
    aggregated_sentiment_meta = sentiment_aggregator(df_meta_vader, title=title, type=type)
    aggregated_sentiment_tsla = sentiment_aggregator(df_tsla_vader, title=title, type=type)
    aggregated_sentiment_world = sentiment_aggregator(df_world_vader, title=title, type=type)
    aggregated_sentiment_politics = sentiment_aggregator(df_politics_vader, title=title, type=type)
    aggregated_sentiment_coronavirus = sentiment_aggregator(df_coronavirus_vader, title=title, type=type)

    lst = [aggregated_sentiment_aapl, aggregated_sentiment_meta, aggregated_sentiment_tsla, aggregated_sentiment_world, aggregated_sentiment_politics, aggregated_sentiment_coronavirus]
    keys = ["AAPL", "META", "TSLA", "World", "Politics", "Coronavirus"]
    
    return pd.concat(lst, keys=keys, axis=1)


In [154]:
#vader_all()
aggregate_sentiment_all(title=False, type="abs_max")

,AAPL,META,TSLA,World,Politics,Coronavirus
Date,,,,,,
2018-01-31,0.0375,0.0375,NaN,NaN,-0.066,NaN
2018-02-01,0.0355,0.0500,0.0,NaN,NaN,NaN
2018-02-02,0.0000,0.0000,NaN,NaN,NaN,NaN
2018-02-04,0.0000,NaN,NaN,0.0,NaN,NaN
2018-02-05,0.0000,-0.0350,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-08-01,NaN,NaN,NaN,NaN,0.061,NaN
2021-09-05,NaN,NaN,NaN,NaN,0.000,NaN
2022-03-26,NaN,NaN,NaN,NaN,0.190,NaN


In [71]:
df_meta = vader_process(df_meta)
df_meta.head()
df_meta.to_csv('../Data-Processed/News/Stock/meta_vader.csv')

,Date,Title,Text,Processed Title,Processed Text,Sentiment Title,Sentiment Text,Positive Title,Positive Text
0,2022-09-01,Top 5 Things to Watch in Markets in the Week A...,"By doing so, you and %USER_NAME% will not be a...",top 5 things to watch in markets in the week a...,by doing so you and user_name will not be able...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",True,False
1,2022-08-30,Top 5 Things to Watch in Markets in the Week A...,By Liz Moyer\r\nInvesting.com -- U.S. stocks f...,top 5 things to watch in markets in the week a...,by liz moyer\r investingcom us stocks fell ag...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.088, 'neu': 0.912, 'pos': 0.0, 'comp...",True,False
2,2022-08-30,Dow Futures Move Higher As Investors Brace for...,By Yasin Ebrahim\r\nInvesting.com -- The Dow c...,dow futures move higher as investors brace for...,by yasin ebrahim\r investingcom the dow close...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.066, 'neu': 0.843, 'pos': 0.09, 'com...",False,True
3,2022-08-29,China's 20th Communist Party Congress: What yo...,"By doing so, you and %USER_NAME% will not be a...",chinas 20th communist party congress what you ...,by doing so you and user_name will not be able...,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",True,False
4,2022-08-26,Top 5 Things to Watch in Markets in the Week A...,By Yasin Ebrahim\r\nInvesting.com -- The S&P 5...,top 5 things to watch in markets in the week a...,by yasin ebrahim\r investingcom the sp 500 sl...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.0, 'neu': 0.906, 'pos': 0.094, 'comp...",True,True


In [77]:
df_tsla = vader_process(df_tsla)
df_tsla.head()
#df_tsla.to_csv('../Data-Processed/News/Stock/tsla_vader.csv')

,Date,Title,Text,Processed Title,Processed Text,Sentiment Title,Sentiment Text,Positive Title,Positive Text,Negative,Neutral,Positive,Compound,Polarity_formula
0,2022-09-01,Top 5 Things to Watch in Markets in the Week A...,By Michael Elkins\r\nA bill targeting Tesla's ...,top 5 things to watch in markets in the week a...,by michael elkins\r a bill targeting teslas na...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",True,False,-0.0,1.0,0.0,0.0,-0.0
1,2022-08-31,Dow Futures Move Higher As Investors Brace for...,By Ambar Warrick \r\nInvesting.com-- Major Chi...,dow futures move higher as investors brace for...,by ambar warrick \r investingcom major chinese...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",False,False,-0.0,1.0,0.0,0.0,-0.0
2,2022-08-31,China's 20th Communist Party Congress: What yo...,"By doing so, you and %USER_NAME% will not be a...",chinas 20th communist party congress what you ...,by doing so you and user_name will not be able...,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",True,False,-0.0,1.0,0.0,0.0,-0.0
3,2022-08-30,Top 5 Things to Watch in Markets in the Week A...,"By doing so, you and %USER_NAME% will not be a...",top 5 things to watch in markets in the week a...,by doing so you and user_name will not be able...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",True,False,-0.0,1.0,0.0,0.0,-0.0
4,2022-08-29,Top 5 Things to Watch in Markets in the Week A...,By Sam Boughedda\r\nA Deutsche Bank analyst ma...,top 5 things to watch in markets in the week a...,by sam boughedda\r a deutsche bank analyst mai...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",True,False,-0.0,1.0,0.0,0.0,-0.0


In [79]:
df_world = vader_process(df_world)
df_world.head()
df_world.to_csv('../Data-Processed/News/Global/world_vader.csv')

In [80]:
df_politics = vader_process(df_politics)
df_politics.head()
df_politics.to_csv('../Data-Processed/News/Global/politics_vader.csv')

In [81]:
df_coronavirus = vader_process(df_coronavirus)
df_coronavirus.head()
df_coronavirus.to_csv('../Data-Processed/News/Global/coronavirus_vader.csv')

,Date,Title,Text,Processed Title,Processed Text,Sentiment Title,Sentiment Text,Positive Title,Positive Text
0,2022-04-24,Top 5 Things to Watch in Markets in the Week A...,By Gina Lee\r\nInvesting.com -- Oil was mixed ...,top 5 things to watch in markets in the week a...,by gina lee\r investingcom oil was mixed on t...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",True,False
1,2022-03-23,Top 5 Things to Watch in Markets in the Week A...,By Yasin Ebrahim\r\nInvesting.com -- U.S. crud...,top 5 things to watch in markets in the week a...,by yasin ebrahim\r investingcom us crude stoc...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.258, 'neu': 0.669, 'pos': 0.073, 'co...",True,False
2,2021-12-13,Top 5 Things to Watch in Markets in the Week A...,By Alistair Smout\r\nLONDON (Reuters) -Booster...,top 5 things to watch in markets in the week a...,by alistair smout\r london reuters booster cov...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'comp...",True,True
3,2021-12-03,Top 5 Things to Watch in Markets in the Week A...,By Trevor Hunnicutt\r\nWASHINGTON (Reuters) - ...,top 5 things to watch in markets in the week a...,by trevor hunnicutt\r washington reuters the ...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.0, 'neu': 0.899, 'pos': 0.101, 'comp...",True,True
4,2021-11-11,Top 5 Things to Watch in Markets in the Week A...,By David Shepardson\r\nWASHINGTON (Reuters) - ...,top 5 things to watch in markets in the week a...,by david shepardson\r washington reuters the ...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",True,False


In [111]:
aggregated_sentiment_aapl = sentiment_aggregator(df_aapl, title=False, type="abs_max")
aggregated_sentiment_meta = sentiment_aggregator(df_meta, title=False, type="abs_max")
aggregated_sentiment_tsla = sentiment_aggregator(df_tsla, title=False, type="abs_max")
aggregated_sentiment_world = sentiment_aggregator(df_world, title=False, type="abs_max")
aggregated_sentiment_politics = sentiment_aggregator(df_politics, title=False, type="abs_max")
aggregated_sentiment_coronavirus = sentiment_aggregator(df_coronavirus, title=False, type="abs_max")
aggregated_sentiment_coronavirus


Date
2020-03-31    0.122000
2020-04-01    0.112667
2020-04-03   -0.089333
2020-04-05   -0.125000
2020-04-06   -0.026500
                ...   
2021-11-11    0.000000
2021-12-03    0.101000
2021-12-13    0.084000
2022-03-23   -0.258000
2022-04-24    0.000000
Name: Sentiment, Length: 74, dtype: float64

In [95]:
 pd.concat([aggregated_sentiment_aapl, aggregated_sentiment_meta, aggregated_sentiment_tsla], axis=1)

,Polarity_formula,Polarity_formula,Polarity_formula
Date,,,
2018-01-31,0.037500,0.0375,NaN
2018-02-01,0.035500,0.0500,0.00000
2018-02-02,0.000000,0.0000,NaN
2018-02-03,NaN,0.0000,NaN
2018-02-04,0.000000,NaN,NaN
...,...,...,...
2022-08-27,NaN,NaN,0.00000
2022-08-29,0.074333,0.0000,0.05575
2022-08-30,-0.028000,0.0010,0.00000
